# LangChain Cookbook 👨‍🍳👩‍🍳

*This cookbook is based off the [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)*

**Goal:** Provide an introductory understanding of the components and use cases of LangChain via [ELI5](https://www.dictionary.com/e/slang/eli5/#:~:text=ELI5%20is%20short%20for%20%E2%80%9CExplain,a%20complicated%20question%20or%20problem.) examples and code snippets. For use cases check out part 2 (coming soon).


**Links:**
* [LC Conceptual Documentation](https://docs.langchain.com/docs/)
* [LC Python Documentation](https://python.langchain.com/en/latest/)
* [LC Javascript/Typescript Documentation](https://js.langchain.com/docs/)
* [LC Discord](https://discord.gg/6adMQxSpJS)
* [www.langchain.com](https://langchain.com/)
* [LC Twitter](https://twitter.com/LangChainAI)


### **What is LangChain?**
> LangChain is a framework for developing applications powered by language models.

**~~TL~~DR**: LangChain makes the complicated parts of working & building with AI models easier. It helps do this in two ways:

1. **Integration** - Bring external data, such as your files, other applications, and api data, to your LLMs
2. **Agency** - Allow your LLMs to interact with it's environment via decision making. Use LLMs to help decide which action to take next

### **Why LangChain?**
1. **Components** - LangChain makes it easy to swap out abstractions and components necessary to work with language models.

2. **Customized Chains** - LangChain provides out of the box support for using and customizing 'chains' - a series of actions strung together.

3. **Speed 🚢** - This team ships insanely fast. You'll be up to date with the latest LLM features.

4. **Community 👥** - Wonderful discord and community support, meet ups, hackathons, etc.

Though LLMs can be straightforward (text-in, text-out) you'll quickly run into friction points that LangChain helps with once you develop more complicated applications.

*Note: This cookbook will not cover all aspects of LangChain. It's contents have been curated to get you to building & impact as quick as possible. For more, please check out [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)*

In [ ]:
openai_api_key=''


In [ ]:
!pip install tiktoken langchain openai faiss-gpu

### **Retrievers**
Easy way to combine documents with language models.

There are many different types of retrievers, the most widely supported is the VectoreStoreRetriever

In [ ]:
! wget https://raw.githubusercontent.com/seiching/purchase/main/lawfaq.txt

--2023-09-19 09:22:41--  https://raw.githubusercontent.com/seiching/purchase/main/lawfaq.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36925 (36K) [text/plain]
Saving to: ‘lawfaq.txt.3’

lawfaq.txt.3        100%[===================>]  36.06K  --.-KB/s    in 0s      

2023-09-19 09:22:41 (111 MB/s) - ‘lawfaq.txt.3’ saved [36925/36925]



In [ ]:
!ls

lawfaq.txt  lawfaq.txt.1  lawfaq.txt.2	lawfaq.txt.3  sample_data


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
filename="lawfaq.txt"
loader = TextLoader(filename)

documents = loader.load()

In [ ]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key,model="text-embedding-ada-002")

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

In [ ]:
# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":6})

from langchain.chains import RetrievalQA, ConversationalRetrievalChain
qa_interface = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key), chain_type="stuff", retriever=retriever, return_source_documents=True)


{'query': '什麼是工程採購?', 'result': '工程採購是指機關或組織為了進行工程項目的建設、設計、施工、維修等活動，通過招標或協商的方式，選擇合適的廠商或承包商，進行相關工程項目的採購和合約簽訂的過程。工程採購涉及到工程項目的規劃、設計、材料供應、施工等各個環節，並需要遵守相關的法律法規和採購程序。', 'source_documents': [Document(page_content='十、工程會訂頒之各類採購契約範本之效力如何？\t政府採購法第63條第1項規定：「各類採購契約以採用主管機關訂定之範本為原則，其要項及內容由主管機關參考國際及國內慣例定之」。', metadata={'source': 'lawfaq.txt'}), Document(page_content='八、統包工程是否得與監造工作併案發包？\t「政府採購法」第24條第2項規定「前項所稱統包，指將工程或財物採購中之設計、施工、供應、安裝或一定期間之維修等併於同一採購契約辦理招標」，並未包括監造。', metadata={'source': 'lawfaq.txt'}), Document(page_content='十三、機關辦理工程採購，其須辦理契約變更而有新增非屬原契約數量清單內所列之工程項目者，其單價如何訂定？\t新增工程項目單價編列方式，應以原預算相關單價分析資料為基礎，並考慮市場價格波動情形；其底價訂定，並應符合政府採購法第46條規定。\n十四、採總價結算之工程採購，實際施作數量較契約所定數量為高時，契約價金如何給付？\t採購契約要項第32點載明：「工程之個別項目實作數量較契約所定數量增減達百分之五以上者，其逾百分之五之部分，變更設計增減契約價金。未達百分之五者，契約價金得不予增減」，機關得將上開內容納入個案契約據以辦理。', metadata={'source': 'lawfaq.txt'}), Document(page_content='一、採購兼有工程、財物、勞務性質，如何認定其採購歸屬？\t採購兼有工程、財物、勞務性質，難以認定其採購歸屬者，依政府採購法第7條第4項規定，按其性質所占預算金額比率最高者歸屬之。\n二、得標廠商於決標後稱其標價書寫錯誤， 拒不簽約，押標金會不會被没收？\t招標文件載明政府採購法第31條第2項第5款者，不發還其押標金。\n

In [ ]:
response = qa_interface("決標有什麼要注意的?")
print(response)

{'query': '決標有什麼要注意的?', 'result': '決標時需要注意以下事項：\n\n1. 決標原則：根據政府採購法第52條第1項，決標原則應符合法定的原則，不能僅以折扣率最高者為決標對象。\n\n2. 契約單價調整：在工程採購中，契約單價的調整應以合理性為前提，並保留雙方協議的機制。\n\n3. 預算限制：廠商報價不得超過預算金額，除非招標文件規定可以進行減價程序。\n\n4. 差額保證金與履約保證金：差額保證金與履約保證金是不同的，廠商在決標後仍需根據招標文件規定繳納履約保證金。\n\n5. 標價一致性：投標廠商填列的總價，大寫和小寫金額應保持一致，若不一致，以文字為準。\n\n6. 招標文件規定：決標時應遵守招標文件中規定的事項，如未明確規定則需依據相關法條進行處理。\n\n7. 標的類別限制：在投標須知中如有限定標的類別的原產地須為該國，則只有國內廠商得標，不違反政府採購法。\n\n8. 預算限制：廠商報價不得超過預算金額，除非招標文件規定可以進行減價程序。\n\n9. 押標金：廠商應使用金融機構支票繳納押標金，並且必須填寫受款人的資訊，否則押標金可能被沒收。\n\n以上是決標時需要注意的一些事項，具體情況還需參考相關法規和招標文件的規定。', 'source_documents': [Document(page_content='十、機關辦理圖書採購，其決標原則，得否以廠商所報書本定價之折扣率最高者為決標對象？\t圖書採購，得以折扣數之高低為決標依據，但須符合政府採購法第52條第1項之決標原則。\n十一、機關辦理財物採購，如有需訂定物價指數調整之條款，得否引用工程採購契約中有關臺灣地區營造工程物價指數計算物價調整款之條款？\t機關辦理財物採購，如於契約中載明契約價金依物價指數調整之約定，應考量標的類別特性，參考工程會訂頒之財物採購契約範本第5條妥為訂定，勿任意引用臺灣地區營造工程物價指數。如欲引用上開工程物價指數，以財物採購案中，其性質屬工程項目者為限，並於招標文件中預為載明適用之項目。', metadata={'source': 'lawfaq.txt'}), Document(page_content='十一、機關辦理工程採購，可否無預算即先決標，或任意挪用預算？\t機關辦理工程採購，如發生無預算即先決標，或預算遭挪用致無款項及時支付廠

In [ ]:
print(response['result'])

決標時需要注意以下事項：

1. 決標原則：根據政府採購法第52條第1項，決標原則應符合法定的原則，不能僅以折扣率最高者為決標對象。

2. 契約單價調整：在工程採購中，契約單價的調整應以合理性為前提，並保留雙方協議的機制。

3. 預算限制：廠商報價不得超過預算金額，除非招標文件規定可以進行減價程序。

4. 差額保證金與履約保證金：差額保證金與履約保證金是不同的，廠商在決標後仍需根據招標文件規定繳納履約保證金。

5. 標價一致性：投標廠商填列的總價，大寫和小寫金額應保持一致，若不一致，以文字為準。

6. 招標文件規定：決標時應遵守招標文件中規定的事項，如未明確規定則需依據相關法條進行處理。

7. 標的類別限制：在投標須知中如有限定標的類別的原產地須為該國，則只有國內廠商得標，不違反政府採購法。

8. 預算限制：廠商報價不得超過預算金額，除非招標文件規定可以進行減價程序。

9. 押標金：廠商應使用金融機構支票繳納押標金，並且必須填寫受款人的資訊，否則押標金可能被沒收。

以上是決標時需要注意的一些事項，具體情況還需參考相關法規和招標文件的規定。
